In [0]:
%sql
-- Set environment to data ingestion site
use catalog bronze;

-- 1. Standardized Consumer
create or replace temp view standardized_consumer as
select
    try_cast(NULLIF(trim(con.consumerid), '') as bigint) as consumer_id,
    initcap(trim(NULLIF(con.Sex, ''))) as sex,
    initcap(trim(NULLIF(con.ethnicity, ''))) as ethnicity,
    initcap(trim(NULLIF(con.Race, ''))) as race,
    try_cast(NULLIF(trim(con.Age), '') as bigint) as age,
    row_number() over (
        partition by try_cast(NULLIF(trim(con.consumerid), '') as bigint)
        order by try_cast(NULLIF(trim(con.Age), '') as bigint) desc
    ) as rnk
from consumer_bronze con
where NULLIF(trim(con.consumerid), '') is not null;

-- 2. Validated Consumer
create or replace temp view validated_consumer as
select
    c.consumer_id,
    c.sex,
    c.ethnicity,
    c.race,
    c.age
from standardized_consumer c
where c.rnk = 1
    and c.age < 130
    and c.age >= 0
    and c.sex in ('Female', 'Male');

-- 3. Standardized Purchase
create or replace temp view standardized_purchase as
select
    try_cast(NULLIF(trim(pur.consumerid), '') as bigint) as consumer_id,
    try_cast(NULLIF(trim(pur.purchaseid), '') as bigint) as purchase_id,
    try_cast(NULLIF(trim(pur.graphed_date), '') as date) as graphed_date,
    try_cast(NULLIF(trim(pur.avocado_bunch_id), '') as bigint) as avocado_bunch_id,
    try_cast(NULLIF(trim(pur.reporting_year), '') as bigint) as reporting_year,
    initcap(trim(NULLIF(pur.`QA process`, ''))) as qa_process,
    try_cast(NULLIF(trim(pur.billing_provider_sku), '') as bigint) as billing_provider_sku,
    try_cast(NULLIF(trim(pur.grocery_store_id), '') as bigint) as grocery_store_id,
    try_cast(NULLIF(trim(pur.price_index), '') as double) as price_index,
    row_number() over (
        partition by try_cast(NULLIF(trim(pur.purchaseid), '') as bigint)
        order by try_cast(NULLIF(trim(pur.graphed_date), '') as date) desc
    ) as rnk
from purchase_bronze pur
where NULLIF(trim(pur.purchaseid), '') is not null
  and NULLIF(trim(pur.consumerid), '') is not null;

-- 4. Validated Purchase
create or replace temp view validated_purchase as
select
    p.consumer_id,
    p.purchase_id,
    p.graphed_date,
    p.avocado_bunch_id,
    p.reporting_year,
    p.qa_process,
    p.billing_provider_sku,
    p.grocery_store_id,
    p.price_index
from standardized_purchase p
inner join validated_consumer c on p.consumer_id = c.consumer_id
where p.rnk = 1;

-- 5. Standardized Fertilizer
create or replace temp view standardized_fertilizer as
select
    try_cast(NULLIF(trim(fer.purchaseid), '') as bigint) as purchase_id,
    try_cast(NULLIF(trim(fer.consumerid), '') as bigint) as consumer_id,
    try_cast(NULLIF(trim(fer.fertilizerid), '') as bigint) as fertilizer_id,
    initcap(trim(NULLIF(fer.type, ''))) as fertilizer_type,
    try_cast(NULLIF(trim(fer.mg), '') as bigint) as mg,
    trim(NULLIF(fer.frequency, '')) as frequency,
    row_number() over (
        partition by try_cast(NULLIF(trim(fer.fertilizerid), '') as bigint)
        order by try_cast(NULLIF(trim(fer.purchaseid), '') as bigint) desc
    ) as rnk
from fertilizer_bronze fer
where NULLIF(trim(fer.purchaseid), '') is not null
  and NULLIF(trim(fer.consumerid), '') is not null
  and NULLIF(trim(fer.fertilizerid), '') is not null;

-- 6. Validated Fertilizer
create or replace temp view validated_fertilizer as
select
    f.purchase_id,
    f.consumer_id,
    f.fertilizer_id,
    f.fertilizer_type,
    f.mg,
    f.frequency
from standardized_fertilizer f
inner join validated_purchase p on f.purchase_id = p.purchase_id
inner join validated_consumer c on f.consumer_id = c.consumer_id
and f.fertilizer_type in (
    'Heavy Metal',
    'Organic',
    'Inorganic'
    )
where f.rnk = 1;

-- 7. Standardized Avocado
create or replace temp view standardized_avocado as
select
    try_cast(NULLIF(trim(avo.consumerid), '') as bigint) as consumer_id,
    try_cast(NULLIF(trim(avo.purchaseid), '') as bigint) as purchase_id,
    try_cast(NULLIF(trim(avo.avocado_bunch_id), '') as bigint) as avocado_bunch_id,
    try_cast(NULLIF(trim(avo.PLU), '') as bigint) as PLU,
    try_cast(NULLIF(trim(avo.`ripe index when picked`), '') as bigint) as avocado_ripe_index,
    try_cast(NULLIF(trim(avo.born_date), '') as date) as born_date,
    try_cast(NULLIF(trim(avo.picked_date), '') as date) as picked_date,
    try_cast(NULLIF(trim(avo.sold_date), '') as date) as sold_date,
    row_number() over (
        partition by try_cast(NULLIF(trim(avo.purchaseid), '') as bigint)
        order by try_cast(NULLIF(trim(avo.born_date), '') as date) desc
    ) as rnk
from avocado_bronze avo
where NULLIF(trim(avo.purchaseid), '') is not null
  and NULLIF(trim(avo.consumerid), '') is not null;

-- 8. Validated Avocado
create or replace temp view validated_avocado as
select
    a.consumer_id,
    a.purchase_id,
    a.avocado_bunch_id,
    a.PLU,
    a.avocado_ripe_index,
    a.born_date,
    a.picked_date,
    a.sold_date
from standardized_avocado a
inner join validated_purchase p on a.purchase_id = p.purchase_id
inner join validated_consumer c on a.consumer_id = c.consumer_id
where a.rnk = 1
    and a.picked_date >= a.born_date
    and a.sold_date >= a.picked_date
;
-- ========================================================
-- 9. GOLD VIEW FOR FINAL OUTPUT
-- ========================================================
create or replace temp view gold_output as
select distinct
    c.consumer_id,
    c.sex,
    c.age,
    a.avocado_bunch_id,
    datediff(a.sold_date, a.born_date) as avocado_days_sold,        -- Avocado age in days
    a.avocado_ripe_index,                                           -- 0-10
    datediff(a.picked_date, a.born_date) as avocado_days_picked,    -- Picked age in days
    f.fertilizer_type
from validated_consumer c
left join validated_purchase p on c.consumer_id = p.consumer_id
left join validated_avocado a on p.purchase_id = a.purchase_id
left join validated_fertilizer f on p.purchase_id = f.purchase_id
;
select 
    gold.* 
from gold_output gold
order by gold.consumer_id